### **Импорт библиотек**

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
!pip install catboost
!pip install tqdm
!pip install pymorphy2[fast]
!pip install fasttext

In [ ]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
import catboost as ctb
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import RidgeClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, SpectralClustering

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from pymorphy2 import MorphAnalyzer
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

import gensim
import gensim.downloader as api
from gensim.test.utils import get_tmpfile
import fasttext
from gensim.models import FastText, KeyedVectors
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### **Считываем и смотрим на данные**

In [ ]:
labled_train_data = pd.read_csv('/content/drive/MyDrive/labled_train_data.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_comments = pd.read_csv('/content/drive/MyDrive/labled_train_comments.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)
labled_train_speed = pd.read_csv('/content/drive/MyDrive/labled_train_tracks_speed.csv', comment='#', sep=',')
labled_train_tracks = pd.read_csv('/content/drive/MyDrive/labled_train_tracks.csv', comment='#', sep='\t').drop('Unnamed: 0', axis=1)

In [ ]:
labled_train_data.head()

,order_id,driver_id,client_id,dttm,date,arrived_distance,arrived_duration,distance,duration,from_latitude,from_longitude,to_latitude,to_longitude,mark,client_rate_ride,client_rides_cnt,driver_rides_cnt,comment,is_aggressive
0,6a0f322ade1a05e5c4cec4344efbce8b,f7c2b293ef94420f5e51abae6889b83b,3156d05c6458a8228bed59f02075a61e,2021-01-22 21:53:00,2021-01-22,150.0,1.0,11.8,20.5,55.795900,37.560300,55.716502,37.524627,Kia K5,NaN,16.0,39.0,Больше нечего сказать,0
1,934ecbe5845426fd3f8ef7938cce2a11,01d029c42c99581080a60679fca06ff9,3156d05c6458a8228bed59f02075a61e,2021-01-24 14:09:00,2021-01-24,570.0,1.0,17.2,26.2,55.716502,37.524627,55.808253,37.638847,Volkswagen Polo,NaN,18.0,338.0,Да,0
2,5348cb339b63eaea3b2cb57a064ce550,3c88deb7df7a73a24ebc229db9783405,3156d05c6458a8228bed59f02075a61e,2021-01-26 21:02:00,2021-01-26,40.0,0.0,5.8,10.9,55.716637,37.524223,55.741958,37.568172,MercedesBenz EClass,NaN,17.0,162.0,Больше нечего сказать,0
3,309ef91c3b51e27d097642169576f67b,f35a8ff85f2095755f16bba91035fbdc,3156d05c6458a8228bed59f02075a61e,2021-01-27 17:24:00,2021-01-27,140.0,1.0,4.5,10.7,55.689076,37.491089,55.716502,37.524627,Kia Optima,NaN,18.0,136.0,Больше нечего сказать,0
4,3506e04e45d39c6e3033637389da1041,0a227ac8d702170c03acf36d55e60d0d,3156d05c6458a8228bed59f02075a61e,2021-01-29 15:31:00,2021-01-29,150.0,1.0,15.4,25.1,55.655489,37.616629,55.716502,37.524627,Kia Rio,5.0,20.0,194.0,Больше нечего сказать,0


In [ ]:
labled_train_comments.loc[labled_train_comments['driver_id']== '469bbb0b9b7883f9df5924326d868d50'].head()

,driver_id,comment
0,469bbb0b9b7883f9df5924326d868d50,Суперски водитель...
4519,469bbb0b9b7883f9df5924326d868d50,Отличный водитель
9941,469bbb0b9b7883f9df5924326d868d50,водитель начал поездку не выключив платное ожи...
14627,469bbb0b9b7883f9df5924326d868d50,спасибо за утренний позитивный настрой
17834,469bbb0b9b7883f9df5924326d868d50,я отменила поездку а с меня списались деньги я...


In [ ]:
labled_train_tracks.head()

,driver_id,dt,lat_,lon_,order_id
0,5947356660903834a6bc52215fe9ffb2,2021-04-06 21:20:16,55.697662,37.562592,e0437da04323f57500e08d6ce7e8372f
1,5947356660903834a6bc52215fe9ffb2,2021-04-06 21:23:19,55.706867,37.585470,e0437da04323f57500e08d6ce7e8372f
2,5947356660903834a6bc52215fe9ffb2,2021-04-06 21:16:52,55.687419,37.544892,e0437da04323f57500e08d6ce7e8372f
3,5947356660903834a6bc52215fe9ffb2,2021-04-06 21:12:06,55.675340,37.540472,e0437da04323f57500e08d6ce7e8372f
4,5947356660903834a6bc52215fe9ffb2,2021-04-06 21:10:03,55.672449,37.543454,e0437da04323f57500e08d6ce7e8372f


In [ ]:
labled_speed.head()

,driver_id,dt,lat_,lon_,order_id,speed,is_aggressive
0,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:43:41,55.792710,37.545409,001662da857b5a39bb402aacf3145f86,29.051632,0.0
1,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:44:40,55.792013,37.544481,001662da857b5a39bb402aacf3145f86,5.906441,0.0
2,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:00,55.791365,37.543695,001662da857b5a39bb402aacf3145f86,15.696000,0.0
3,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:20,55.791267,37.543512,001662da857b5a39bb402aacf3145f86,2.862000,0.0
4,b76545fa3cc14acd6a69ac13c1edac33,2021-02-09 21:45:37,55.791175,37.543634,001662da857b5a39bb402aacf3145f86,2.710588,0.0


### **Модель**

In [ ]:
X, y = labled_train_data[labled_train_data.columns[:-1]], labled_train_data['is_aggressive']

In [ ]:
def doc_to2_vec_clustering(X, y):
  # stop_words = set(stopwords.words('russian')) # стоп-слова из nltk
  # stop_words.add('')
  # stop_words.add(' ')
  # stop_words.add('\t')
  # dataset = X.join(y).copy()
  # dataset = dataset.loc[dataset['is_aggressive'] == 1]
  # data = []
  # tag_n = 0
  # for row in dataset.itertuples(): # перебираем все строки в датасете
  #     if getattr(row, 'comment'):
  #       comment = getattr(row, 'comment')
  #       comment = [lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0] for word in comment.split(' ') if lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0] not in stop_words and lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0].isalpha()]
  #       data.append(TaggedDocument(comment, [tag_n]))
  #       tag_n += 1
  # model = Doc2Vec(data, min_count=1, size=10, window=4, workers=-1, seed=42, epohs=20)
  # print('MODEL D2V TRAINED')
  # vectors = []
  # for row in dataset.itertuples(): # перебираем все строки в датасете
  #     if getattr(row, 'comment'):
  #       comment = getattr(row, 'comment')
  #       comment = [lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0] for word in comment.split(" ") if lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0] not in stop_words and lemmatize_and_word_class(re.sub(r'\W', '', word.lower()))[0].isalpha()]
  #       print(comment)
  #       vectors.append(model.infer_vector(comment))
  # return vectors, model
  pass

In [ ]:
vectors, model = doc_to_vec_clustering(X, y)


In [ ]:
word='a1_2+!'
word = re.sub(r'[\d\W]', '', word).lower().strip()
print(word)

a_


In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42).fit(vectors)
docvec = kmeans.cluster_centers_[2]  # assuming such a doc-tag exists
print(docvec)
similar_words = model.most_similar(positive=[docvec])
print(similar_words)

In [70]:
class Model:
  def __init__(self):
    self.model = None
    self.text_vectorizer = None
    self.clustering_model = None

    self.aggressive_words = {'verb': set(), 'adj': set(), 'all_words': set(), 'noun': set()}
    self.morph_analyzer = MorphAnalyzer()

    self.stop_words = set(stopwords.words('russian')) # стоп-слова из nltk
    self.stop_words.add('')
    self.stop_words.add(' ')
    self.stop_words.add('\t')

  # возвращает нормальную форму слова(при normal_form=True, иначе просто слово) и его тэг(характеристики слова)
  def word_preprocess(self, word, word_normal_form=False):
      word = re.sub(r'[\d\W]', '', word).lower().strip() # убирает пробелы, цифры и знаки препинания
      word = word.replace('_', '')
      w = self.morph_analyzer.parse(word)[0]
      if word_normal_form:
        return w.normal_form, w.tag
      return word, w.tag

  # тренировка кластеризатора текстов
  def cluster_model_train(self, vectors):
    self.clustering_model = KMeans(n_clusters=2, random_state=42, n_init=20)
    self.clustering_model.fit(vectors)

  # получаем косинусное расстояние для вектора коммента и центров кластеров
  def cluster_features(self, text):
    similarity = []
    clusters = self.clustering_model.cluster_centers_
    text = [self.word_preprocess(word, word_normal_form=True)[0] for word in text.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
    text = [word for word in text if word != '']
    if len(text) == 0:
      for i in range(len(clusters)):
        similarity.append([0,0,0])
      return similarity
    vector = self.text_vectorizer.infer_vector(text)
    similarity = []
    for i in range(len(clusters)):
      similarity.append([euclidean(vector, clusters[i]), self.clustering_model.predict([vector])[0]])
    return similarity

  def train_doc2vec_model(self, X, y):
    print('training vectorizer model...')
    dataset = X.join(y).copy()
    # dataset = dataset.loc[dataset['is_aggressive'] == 1]
    data = []
    tag_n = 0
    for row in dataset.itertuples(): # перебираем все строки в датасете
        if getattr(row, 'comment'):
          comment = getattr(row, 'comment')
          comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
          comment = [word for word in comment if word != '']
          if len(comment) > 0:
            data.append(TaggedDocument(comment, [tag_n]))
            tag_n += 1
    self.text_vectorizer = Doc2Vec(data, min_count=1, size=30, window=4, workers=-1, seed=42, epohs=15)
    print('TEXT WECTORIZER TRAINED')

    print('training clustering model...')
    vectors = []
    for row in dataset.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'comment'):
        comment = getattr(row, 'comment')
        comment = [self.word_preprocess(word, word_normal_form=True)[0] for word in comment.split(" ") if self.word_preprocess(word, word_normal_form=True)[0] not in self.stop_words]
        comment = [word for word in comment if word != '']
        if len(comment) > 0:
          vectors.append(self.text_vectorizer.infer_vector(comment))
    self.cluster_model_train(vectors)
    print('CLUSTERING COMPLETED')

  # средний рейтинг по комментариям для каждого водителя  (плохо работает, хз че с ними делать), не юзать пока
  def mean_comments_aggressive_rate(self, comm_dataset, X):
    mean_comments_aggressive_rate = []
    driver_ids = X['driver_id']
    for driver_id in driver_ids:
      driver_comments_rate = []
      driver_comments = comm_dataset.loc[comm_dataset['driver_id'] == driver_id]
      for comment in driver_comments:
        driver_comments_rate.append(self.get_cosine_sim_to_clusters(comment)[2])
      mean_comments_aggressive_rate.append(np.mean(driver_comments_rate))
    return mean_comments_aggressive_rate

  # добавляет слова в словарь агрессивных слов
  def fill_agressive_vocab(self, X, y):
    dataset = X.join(y).copy()
    for row in dataset.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'is_aggressive') == 1 and getattr(row, 'comment'):
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        for word in words:
          if word[0] not in self.stop_words:
            if 'VERB' in word[1]:
              self.aggressive_words['verb'].add(word[0])
            elif 'ADJF' in word[1] or 'ADJS' in word[1]:
              self.aggressive_words['adj'].add(word[0])
            elif 'NOUN' in word[1]:
              self.aggressive_words['noun'].add(word[0])


  # делаем NLP фичи на основе сгенерированного словаря
  def NLP_feature_extract(self, X, y=None):
    agg_verbs_rate = [] # глаголы
    agg_adjs_rate = [] # прилагательные
    agg_nouns_rate = [] # существительные
    for row in X.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'comment'):
        words = [self.word_preprocess(word, word_normal_form=True) for word in getattr(row, 'comment').split(' ')]
        words_verb = [word[0] for word in words if 'VERB' in word[1] and word[0] not in self.stop_words]
        words_adj = [word[0] for word in words if ('ADJF' in word[1] or 'ADJS' in word[1]) and word[0] not in self.stop_words]
        words_noun = [word[0] for word in words if 'NOUN' in word[1] and word[0] not in self.stop_words]

        if len(words_verb) > 0:
          agg_verbs_rate.append(len(set(words_verb) & self.aggressive_words['verb']) / len(words_verb))
        else:
          agg_verbs_rate.append(0)

        if len(words_adj) > 0:
          agg_adjs_rate.append(len(set(words_adj) & self.aggressive_words['adj']) / len(words_adj))
        else:
          agg_adjs_rate.append(0)

        if len(words_noun) > 0:
          agg_nouns_rate.append(len(set(words_noun) & self.aggressive_words['noun']) / len(words_noun))
        else:
          agg_nouns_rate.append(0)

    return agg_verbs_rate, agg_adjs_rate, agg_nouns_rate

  # отбор фичей
  def features(self, X, comm_dataset=None):
    data = X.copy()
    agg_verbs_rate, agg_adjs_rate, agg_nouns_rate = self.NLP_feature_extract(data)

    data['agg_verbs_rate'] = agg_verbs_rate
    data['agg_adjs_rate'] = agg_adjs_rate
    data['agg_nouns_rate'] = agg_nouns_rate

    feature_list = ['agg_verbs_rate', 'agg_adjs_rate', 'agg_nouns_rate']

    if comm_dataset is not None:
      data['mean_comments_aggressive_rate'] = self.mean_comments_aggressive_rate(comm_dataset, X)
      feature_list.append('mean_comments_aggressive_rate')


    similarities = []
    for row in data.itertuples(): # перебираем все строки в датасете
      if getattr(row, 'comment'):
        comment = getattr(row, 'comment')
        similarities.append(self.cluster_features(comment))

    for cluster in range(len(self.clustering_model.cluster_centers_)):
      data[f"euclidean_cluster_{cluster + 1}"] = [similarities[i][cluster][0] for i in range(len(similarities))]
      feature_list.append(f"euclidean_cluster_{cluster + 1}")
    data['cluster'] = [similarities[i][0][1] for i in range(len(similarities))]
    feature_list.append('cluster')

    # data['is_comment'] = [1 if getattr(row, 'comment') else 0 for row in data.itertuples()]
    # feature_list.append('is_comment')

    # заполним NaN средними значениями
    for feature in feature_list:
      data = data.fillna({feature: data[feature].mean()})
    return data[feature_list]

  # кросс-валидация и предикт на тесте
  def train_eval(self, X, y, comm_dataset=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

    self.fill_agressive_vocab(X_train, y_train)
    self.train_doc2vec_model(X_train, y_train)

    # self.model = ctb.CatBoostClassifier(random_state=42, iterations=100) # пробовал catboost для интереса
    self.model = LogisticRegression(random_state=42)

    cv_score = cross_val_score(self.model, self.features(X_train, comm_dataset), y_train, cv=5, scoring='roc_auc')
    
    self.model.fit(self.features(X_train, comm_dataset), y_train)

    print('Test Roc-Auc score:', roc_auc_score(y_test, self.model.predict_proba(self.features(X_test, comm_dataset))[:, 1]))
    print('Train Roc-Auc score:', roc_auc_score(y_train, self.model.predict_proba(self.features(X_train, comm_dataset))[:, 1]))
    print(f"CV_mean roc_auc: {np.mean(cv_score)}, CV_folds_score: {cv_score}")
  
  def predict(self, X):
    return self.model.predict(X)

In [71]:
model = Model()
model.train_eval(X, y)

training vectorizer model...
TEXT WECTORIZER TRAINED
training clustering model...
CLUSTERING COMPLETED
Test Roc-Auc score: 0.6668262373239023
Train Roc-Auc score: 0.6838763867242741
CV_mean roc_auc: 0.6819522032387957, CV_folds_score: [0.62861001 0.74803329 0.66013863 0.70424985 0.66872923]


### **Отсчет по фичам**

Скор без NLP фич - 0.5644
Скор с NLP фичами:

  - эмпирически: 0.6965 (если добавить все слова, а не по типу слов - результат хуже на 0.005) CV_mean: 0.7213110197676796, CV_std: 0.036754591689793115 - глаг+прил+сущ (но словарь сформирован для данных, на которых и идет предсказание, поэтому мб и такой скор)

  - кластеризация и семантическая модель (doc2vec): 0.640-651 (фичи тип кластера, евклидово расстояние до каждого кластера) CV_mean roc_auc: 0.6625926120596789, CV_folds_score: [0.63023583 0.71938044 0.62881106 0.68139226 0.65314347] косинусное расстояние хоть обычно и юзают для текстов, но тут ухудшило результат(странно)
  - эмпирические + кластеризация 0.668 CV_mean roc_auc: 0.6819522032387957, CV_folds_score: [0.62861001 0.74803329 0.66013863 0.70424985 0.66872923]

  